In [ ]:
import tensorflow as tf

In [ ]:
import tensorflow as tf

# Desactivar la ejecución ansiosa
tf.compat.v1.disable_eager_execution()

# Definir las características
batch_size = 128
input_channels = 96
output_channels = 256
kernel_size = (7, 7)
input_size = (40, 40)

# Crear un placeholder para la entrada (batch_size, height, width, channels)
input_placeholder = tf.compat.v1.placeholder(tf.float32, shape=(batch_size, input_size[0], input_size[1], input_channels))

# Definir la capa convolucional
conv_layer = tf.keras.layers.Conv2D(filters=output_channels, kernel_size=kernel_size, strides=(1, 1), padding='valid', activation=tf.nn.relu)

# Pasar la entrada a través de la capa convolucional
output = conv_layer(input_placeholder)

# Inicializar TensorFlow
sess = tf.compat.v1.Session()
sess.run(tf.compat.v1.global_variables_initializer())

# Simular datos de entrada
import numpy as np
input_data = np.random.rand(batch_size, input_size[0], input_size[1], input_channels)

# Ejecutar la sesión para obtener la salida
output_data = sess.run(output, feed_dict={input_placeholder: input_data})

# Imprimir la forma de la salida
print("Shape of the output:", output_data.shape)


Shape of the output: (128, 34, 34, 256)


In [1]:
import numpy as np
import random
import tensorflow.compat.v1 as tf
from tensorflow import keras
tf.disable_v2_behavior()
tf.compat.v1.disable_resource_variables()


# Load training and test data (MNIST dataset):
dataLoader = keras.datasets.mnist
(features, labels), (testFeatures, testLabels) = dataLoader.load_data()
onehot_labels = np.zeros((labels.shape[0], 10))
onehot_labels[np.arange(labels.shape[0]), labels] = 1
labels = onehot_labels
features = features
testFeatures = testFeatures
onehot_testLabels = np.zeros((testLabels.shape[0], 10))
onehot_testLabels[np.arange(testLabels.shape[0]), testLabels] = 1
testLabels = onehot_testLabels

# Set the parameters:
NumClasses = 10
BatchLength = 16
Size = [28, 28, 1]
NumIteration = 40001
LearningRate = 1e-4
EvalFreq = 1000
NumKernels = [16, 32, 64]


# spectral pooling size:
#         1/2  - 0
#         6/8 - 1
specPoolSize = 0


def fourier_complex_relu(x):
    real = tf.real(x)
    imag = tf.imag(x)
    return tf.complex(tf.cast(real*real+imag*imag > 0.1, tf.float32)*real, tf.cast(real*real+imag*imag > 0.1, tf.float32)*imag)


def convolution_in_freq_domain_without_ifft(f_input, out_channels):
    in_shape = f_input.get_shape()
    bias_r = tf.get_variable('BiasReal', [out_channels], dtype=tf.float32)
    bias_c = tf.get_variable('BiasComp', [out_channels], dtype=tf.float32)
    bias = tf.complex(bias_r, bias_c)
    # Spectral pooling:
    if specPoolSize == 0:
        f_input = tf.slice(f_input, [0, int(in_shape[1] // 4), int(in_shape[2] // 4), 0], [-1, int(in_shape[1] // 2), int(in_shape[2] // 2), in_shape[-1]])
    elif specPoolSize == 1:
        f_input = tf.slice(f_input, [0, int(in_shape[1] // 8), int(in_shape[2] // 8), 0], [-1, int(in_shape[1]) - int(2 * in_shape[1] // 8), int(in_shape[2]) - int(2 * in_shape[2] // 8), in_shape[-1]])
    in_shape = f_input.get_shape()
    w_r = tf.get_variable('w_r', [int(in_shape[1]), int(in_shape[2]), int(in_shape[3]), out_channels])
    w_i = tf.get_variable('w_i', [int(in_shape[1]), int(in_shape[2]), int(in_shape[3]), out_channels])
    w = tf.complex(w_r, w_i)
    fourier_kernel = w
    fourier_kernel = tf.tile(tf.expand_dims(fourier_kernel, 0), [BatchLength, 1, 1, 1, 1])
    out = []
    for ind in range(out_channels):
        res = tf.multiply(f_input[:, :, :, :], fourier_kernel[:, :, :, :, ind])
        res = tf.add(res, bias[ind])
        res = tf.expand_dims(tf.reduce_sum(res, 3), -1)
        out.append(res)
    out = tf.concat(out, 3)# Con estas líneas utilizando tf.keras.layers.BatchNormalization
    norm_real = tf.keras.layers.BatchNormalization()(tf.real(out), training=True)
    norm_comp = tf.keras.layers.BatchNormalization()(tf.imag(out), training=True)

    out = tf.complex(norm_real, norm_comp)
    out = fourier_complex_relu(out)
    return out


tf.reset_default_graph()
InputData = tf.placeholder(tf.float32, [None] + Size)  # input images
OneHotLabels = tf.placeholder(tf.int32, [None, NumClasses])  # the expected outputs, labels


# Take the input to Fourier domain
CurrentInput = tf.cast(InputData, tf.complex64)
CurrentInput = tf.transpose(CurrentInput, [3, 0, 1, 2])
fourierInput = tf.fft2d(CurrentInput, name=None)
fourierInput = tf.transpose(fourierInput, [1, 2, 3, 0])
fourierInput = tf.roll(fourierInput, shift=[int(Size[0]//2), int(Size[1]//2)], axis=[1, 2])
CurrentFilters = Size[-1]
# a loop which creates all layers
for N in range(len(NumKernels)):
    with tf.variable_scope('conv' + str(N)):
        fourierInput = convolution_in_freq_domain_without_ifft(fourierInput, NumKernels[N])
with tf.variable_scope('FC'):
    fourierInput = tf.square(tf.real(fourierInput)) + tf.square(tf.imag(fourierInput))
    CurrentShape = fourierInput.get_shape()
    FeatureLength = int(CurrentShape[1]*CurrentShape[2]*CurrentShape[3])
    FC = tf.reshape(fourierInput, [-1, FeatureLength])
    W = tf.get_variable('W', [FeatureLength, NumClasses])
    FC = tf.matmul(FC, W)
    Bias = tf.get_variable('Bias', [NumClasses])
    FC = tf.add(FC, Bias)


with tf.name_scope('loss'):
    Loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=OneHotLabels, logits=FC))


with tf.name_scope('optimizer'):
    Optimizer = tf.train.AdamOptimizer(LearningRate).minimize(Loss)


with tf.name_scope('accuracy'):
    CorrectPredictions = tf.equal(tf.argmax(FC, 1), tf.argmax(OneHotLabels, 1))
    Accuracy = tf.reduce_mean(tf.cast(CorrectPredictions, tf.float32))


Init = tf.global_variables_initializer()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
with tf.Session(config=config) as Sess:
    Sess.run(Init)
    Step = 1
    while Step < NumIteration:
        UsedInBatch = random.sample(range(features.shape[0]), BatchLength)
        batch_xs = features[UsedInBatch, :]
        batch_ys = labels[UsedInBatch, :]
        batch_xs = np.reshape(batch_xs, [BatchLength]+Size)
        _, Acc, L = Sess.run([Optimizer, Accuracy, Loss], feed_dict={InputData: batch_xs, OneHotLabels: batch_ys})
        if (Step % 100) == 0:
            print("Iteration: "+str(Step))
            print("Accuracy:" + str(Acc))
            print("Loss:" + str(L))
        if (Step % EvalFreq) == 0:
            SumAcc = 0.0
            for i in range(0, testFeatures.shape[0]):
                batch_xs = testFeatures[i, :]
                batch_ys = testLabels[i, :]
                batch_xs = np.reshape(batch_xs, [1]+Size)
                batch_ys = np.reshape(batch_ys, [1, NumClasses])
                a = Sess.run(Accuracy, feed_dict={InputData: batch_xs, OneHotLabels: batch_ys})
                SumAcc += a
            print("Independent Test set: "+str(float(SumAcc)/testFeatures.shape[0]))
        Step += 1

Instructions for updating:
non-resource variables are not supported in the long term


11490434/11490434 [==============================] - 0s 0us/step


<ipython-input-1-11ac3a6c921d>:66: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  norm_real = tf.layers.batch_normalization(tf.real(out), training=True)
Instructions for updating:
Colocations handled automatically by placer.
<ipython-input-1-11ac3a6c921d>:67: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  norm_comp = tf.layers.batch_normalization(tf.imag(out), training=True)


Iteration: 100
Accuracy:0.75
Loss:0.9307014
Iteration: 200
Accuracy:0.8125
Loss:0.7756984
Iteration: 300
Accuracy:0.6875
Loss:0.93721366
Iteration: 400
Accuracy:0.75
Loss:0.73414516
Iteration: 500
Accuracy:0.75
Loss:1.0891478
Iteration: 600
Accuracy:0.9375
Loss:0.6183495
Iteration: 700
Accuracy:0.625
Loss:0.7580894
Iteration: 800
Accuracy:0.75
Loss:0.69200206
Iteration: 900
Accuracy:0.875
Loss:0.5683291
Iteration: 1000
Accuracy:0.9375
Loss:0.29714042
Independent Test set: 0.7118
Iteration: 1100
Accuracy:0.8125
Loss:0.4009171
Iteration: 1200
Accuracy:0.9375
Loss:0.33134592
Iteration: 1300
Accuracy:0.9375
Loss:0.3102257
Iteration: 1400
Accuracy:0.75
Loss:0.7790923
Iteration: 1500
Accuracy:0.4375
Loss:1.6638623
Iteration: 1600
Accuracy:0.6875
Loss:1.5161495
Iteration: 1700
Accuracy:0.625
Loss:0.862098
Iteration: 1800
Accuracy:0.75
Loss:0.78379583
Iteration: 1900
Accuracy:0.9375
Loss:0.4149748
Iteration: 2000
Accuracy:0.6875
Loss:1.2445016
Independent Test set: 0.7855
Iteration: 2100
Accur

In [2]:
pip install matplotlib

# RECONOCIMIENTO DE DÍGITOS


In [ ]:
import numpy as np
import random
import tensorflow.compat.v1 as tf
import matplotlib.pyplot as plt
from tensorflow import keras

tf.disable_v2_behavior()
tf.compat.v1.disable_resource_variables()

# Load training and test data (MNIST dataset):
dataLoader = keras.datasets.mnist
(features, labels), (testFeatures, testLabels) = dataLoader.load_data()
onehot_labels = np.zeros((labels.shape[0], 10))
onehot_labels[np.arange(labels.shape[0]), labels] = 1
labels = onehot_labels
features = features
testFeatures = testFeatures
onehot_testLabels = np.zeros((testLabels.shape[0], 10))
onehot_testLabels[np.arange(testLabels.shape[0]), testLabels] = 1
testLabels = onehot_testLabels

# Set the parameters:
NumClasses = 10
BatchLength = 16
Size = [28, 28, 1]
NumIteration = 40001
LearningRate = 1e-4
EvalFreq = 1000
NumKernels = [16, 32, 64]

# Spectral pooling size:
#         1/2  - 0
#         6/8 - 1
specPoolSize = 0


def fourier_complex_relu(x):
    real = tf.real(x)
    imag = tf.imag(x)
    return tf.complex(tf.cast(real * real + imag * imag > 0.1, tf.float32) * real,
                      tf.cast(real * real + imag * imag > 0.1, tf.float32) * imag)


def convolution_in_freq_domain_without_ifft(f_input, out_channels):
    in_shape = f_input.get_shape()
    bias_r = tf.get_variable('BiasReal', [out_channels], dtype=tf.float32)
    bias_c = tf.get_variable('BiasComp', [out_channels], dtype=tf.float32)
    bias = tf.complex(bias_r, bias_c)
    # Spectral pooling:
    if specPoolSize == 0:
        f_input = tf.slice(f_input, [0, int(in_shape[1] // 4), int(in_shape[2] // 4), 0],
                           [-1, int(in_shape[1] // 2), int(in_shape[2] // 2), in_shape[-1]])
    elif specPoolSize == 1:
        f_input = tf.slice(f_input, [0, int(in_shape[1] // 8), int(in_shape[2] // 8), 0],
                           [-1, int(in_shape[1]) - int(2 * in_shape[1] // 8), int(in_shape[2]) - int(2 * in_shape[2] // 8),
                            in_shape[-1]])
    in_shape = f_input.get_shape()
    w_r = tf.get_variable('w_r', [int(in_shape[1]), int(in_shape[2]), int(in_shape[3]), out_channels])
    w_i = tf.get_variable('w_i', [int(in_shape[1]), int(in_shape[2]), int(in_shape[3]), out_channels])
    w = tf.complex(w_r, w_i)
    fourier_kernel = w
    fourier_kernel = tf.tile(tf.expand_dims(fourier_kernel, 0), [BatchLength, 1, 1, 1, 1])
    out = []
    for ind in range(out_channels):
        res = tf.multiply(f_input[:, :, :, :], fourier_kernel[:, :, :, :, ind])
        res = tf.add(res, bias[ind])
        res = tf.expand_dims(tf.reduce_sum(res, 3), -1)
        out.append(res)
    out = tf.concat(out, 3)# Con estas líneas utilizando tf.keras.layers.BatchNormalization
    norm_real = tf.keras.layers.BatchNormalization()(tf.real(out), training=True)
    norm_comp = tf.keras.layers.BatchNormalization()(tf.imag(out), training=True)

    out = tf.complex(norm_real, norm_comp)
    out = fourier_complex_relu(out)
    return out


tf.reset_default_graph()
InputData = tf.placeholder(tf.float32, [None] + Size)  # input images
OneHotLabels = tf.placeholder(tf.int32, [None, NumClasses])  # the expected outputs, labels

# Take the input to Fourier domain
CurrentInput = tf.cast(InputData, tf.complex64)
CurrentInput = tf.transpose(CurrentInput, [3, 0, 1, 2])
fourierInput = tf.fft2d(CurrentInput, name=None)
fourierInput = tf.transpose(fourierInput, [1, 2, 3, 0])
fourierInput = tf.roll(fourierInput, shift=[int(Size[0] // 2), int(Size[1] // 2)], axis=[1, 2])
CurrentFilters = Size[-1]

# a loop which creates all layers
for N in range(len(NumKernels)):
    with tf.variable_scope('conv' + str(N)):
        fourierInput = convolution_in_freq_domain_without_ifft(fourierInput, NumKernels[N])

with tf.variable_scope('FC'):
    fourierInput = tf.square(tf.real(fourierInput)) + tf.square(tf.imag(fourierInput))
    CurrentShape = fourierInput.get_shape()
    FeatureLength = int(CurrentShape[1] * CurrentShape[2] * CurrentShape[3])
    FC = tf.reshape(fourierInput, [-1, FeatureLength])
    W = tf.get_variable('W', [FeatureLength, NumClasses])
    FC = tf.matmul(FC, W)
    Bias = tf.get_variable('Bias', [NumClasses])
    FC = tf.add(FC, Bias)

with tf.name_scope('loss'):
    Loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=OneHotLabels, logits=FC))

with tf.name_scope('optimizer'):
    Optimizer = tf.train.AdamOptimizer(LearningRate).minimize(Loss)

with tf.name_scope('accuracy'):
    CorrectPredictions = tf.equal(tf.argmax(FC, 1), tf.argmax(OneHotLabels, 1))
    Accuracy = tf.reduce_mean(tf.cast(CorrectPredictions, tf.float32))

Init = tf.global_variables_initializer()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.compat.v1.Session(config=config) as Sess:
    Sess.run(Init)
    Step = 1
    while Step < NumIteration:
        UsedInBatch = random.sample(range(features.shape[0]), BatchLength)
        batch_xs = features[UsedInBatch, :]
        batch_ys = labels[UsedInBatch, :]
        batch_xs = np.reshape(batch_xs, [BatchLength] + Size)
        _, Acc, L = Sess.run([Optimizer, Accuracy, Loss], feed_dict={InputData: batch_xs, OneHotLabels: batch_ys})
        if (Step % 100) == 0:
            print("Iteration: " + str(Step))
            print("Accuracy:" + str(Acc))
            print("Loss:" + str(L))
        if (Step % EvalFreq) == 0:
            SumAcc = 0.0
            for i in range(0, testFeatures.shape[0]):
                batch_xs = testFeatures[i, :]
                batch_ys = testLabels[i, :]
                batch_xs = np.reshape(batch_xs, [1] + Size)
                batch_ys = np.reshape(batch_ys, [1, NumClasses])
                a = Sess.run(Accuracy, feed_dict={InputData: batch_xs, OneHotLabels: batch_ys})
                SumAcc += a
            print("Independent Test set: " + str(float(SumAcc) / testFeatures.shape[0]))
        Step += 1

    # Después de imprimir la forma de la salida
    print("Forma de la salida:", FC.shape)

    # Visualizar algunas imágenes y sus predicciones
    num_visualize = 5  # Puedes ajustar el número de imágenes a visualizar

    # Seleccionar aleatoriamente algunas muestras del conjunto de prueba
    visualize_indices = np.random.choice(testFeatures.shape[0], num_visualize, replace=False)

    # Crear una sesión sin inicializar explícitamente las variables
    for i in visualize_indices:
        # Obtener la imagen y la etiqueta verdadera
        input_image = testFeatures[i, :]
        true_label = testLabels[i, :]

        # Preprocesar la entrada para que sea compatible con la red neuronal
        input_image = np.reshape(input_image, [1] + Size)

        # Obtener la predicción del modelo
        prediction = np.argmax(Sess.run(FC, feed_dict={InputData: input_image}), axis=1)

        # Visualizar la imagen, la etiqueta verdadera y la predicción
        plt.imshow(np.squeeze(input_image), cmap='gray')
        plt.title(f'Etiqueta real: {np.argmax(true_label)}, Etiqueta predicha: {prediction[0]}')
        plt.show()
        plt.pause(2)  # Pausa de 2 segundos entre imágenes


Iteration: 100
Accuracy:0.8125
Loss:1.7138262
Iteration: 200
Accuracy:0.625
Loss:0.7691761
Iteration: 300
Accuracy:0.6875
Loss:0.89332676
Iteration: 400
Accuracy:0.625
Loss:1.023562
Iteration: 500
Accuracy:0.8125
Loss:0.43535298
Iteration: 600
Accuracy:0.75
Loss:0.65603673
Iteration: 700
Accuracy:0.75
Loss:0.87891245
Iteration: 800
Accuracy:0.5625
Loss:0.672582
Iteration: 900
Accuracy:0.75
Loss:1.0081573
Iteration: 1000
Accuracy:0.5625
Loss:1.0080729
Independent Test set: 0.7258
Iteration: 1100
Accuracy:0.8125
Loss:0.5857743
Iteration: 1200
Accuracy:0.8125
Loss:0.85653293
Iteration: 1300
Accuracy:0.9375
Loss:0.2487207
Iteration: 1400
Accuracy:0.875
Loss:0.9716654
Iteration: 1500
Accuracy:0.625
Loss:0.7343407
Iteration: 1600
Accuracy:0.75
Loss:0.95528203
Iteration: 1700
Accuracy:0.6875
Loss:0.9565917
Iteration: 1800
Accuracy:1.0
Loss:0.23287597
Iteration: 1900
Accuracy:0.6875
Loss:0.80322546
Iteration: 2000
Accuracy:1.0
Loss:0.15903825
Independent Test set: 0.805
Iteration: 2100
Accurac

# ENFERMEDAD

In [ ]:
pip install -U scikit-learn

In [ ]:
import numpy as np
import random
import tensorflow.compat.v1 as tf
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

tf.disable_v2_behavior()
tf.compat.v1.disable_resource_variables()

# Cargar el conjunto de datos de cáncer de mama
data = load_breast_cancer()
features = data.data
labels = data.target

# Dividir el conjunto de datos en entrenamiento y prueba
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Normalizar los datos
scaler = StandardScaler()
features_train = scaler.fit_transform(features_train)
features_test = scaler.transform(features_test)

# Convertir las etiquetas a one-hot encoding (en este caso, solo hay dos clases)
labels_train_onehot = np.zeros((labels_train.shape[0], 2))
labels_train_onehot[np.arange(labels_train.shape[0]), labels_train] = 1

labels_test_onehot = np.zeros((labels_test.shape[0], 2))
labels_test_onehot[np.arange(labels_test.shape[0]), labels_test] = 1

# Set the parameters:
NumClasses = 2  # Se ajusta al número de clases en el conjunto de datos de cáncer de mama
BatchLength = 16
Size = [features.shape[1], 1, 1]  # Ajustar el tamaño de entrada a la cantidad de características
NumIteration = 40001
LearningRate = 1e-4
EvalFreq = 1000
NumKernels = [16, 32, 64]

# Spectral pooling size:
#         1/2  - 0
#         6/8 - 1
specPoolSize = 0


def fourier_complex_relu(x):
    real = tf.real(x)
    imag = tf.imag(x)
    return tf.complex(tf.cast(real * real + imag * imag > 0.1, tf.float32) * real,
                      tf.cast(real * real + imag * imag > 0.1, tf.float32) * imag)


def convolution_in_freq_domain_without_ifft(f_input, out_channels):
    in_shape = f_input.get_shape()
    bias_r = tf.get_variable('BiasReal', [out_channels], dtype=tf.float32)
    bias_c = tf.get_variable('BiasComp', [out_channels], dtype=tf.float32)
    bias = tf.complex(bias_r, bias_c)
    # Spectral pooling:
    if specPoolSize == 0:
        f_input = tf.slice(f_input, [0, int(in_shape[1] // 4), int(in_shape[2] // 4), 0],
                           [-1, int(in_shape[1] // 2), int(in_shape[2] // 2), in_shape[-1]])
    elif specPoolSize == 1:
        f_input = tf.slice(f_input, [0, int(in_shape[1] // 8), int(in_shape[2] // 8), 0],
                           [-1, int(in_shape[1]) - int(2 * in_shape[1] // 8), int(in_shape[2]) - int(2 * in_shape[2] // 8),
                            in_shape[-1]])
    in_shape = f_input.get_shape()
    w_r = tf.get_variable('w_r', [int(in_shape[1]), int(in_shape[2]), int(in_shape[3]), out_channels])
    w_i = tf.get_variable('w_i', [int(in_shape[1]), int(in_shape[2]), int(in_shape[3]), out_channels])
    w = tf.complex(w_r, w_i)
    fourier_kernel = w
    fourier_kernel = tf.tile(tf.expand_dims(fourier_kernel, 0), [BatchLength, 1, 1, 1, 1])
    out = []
    for ind in range(out_channels):
        res = tf.multiply(f_input[:, :, :, :], fourier_kernel[:, :, :, :, ind])
        res = tf.add(res, bias[ind])
        res = tf.expand_dims(tf.reduce_sum(res, 3), -1)
        out.append(res)
    out = tf.concat(out, 3)
    norm_real = tf.keras.layers.BatchNormalization()(tf.real(out), training=True)
    norm_comp = tf.keras.layers.BatchNormalization()(tf.imag(out), training=True)

    out = tf.complex(norm_real, norm_comp)
    out = fourier_complex_relu(out)
    return out


tf.reset_default_graph()
InputData = tf.placeholder(tf.float32, [None] + Size)  # input images
OneHotLabels = tf.placeholder(tf.int32, [None, NumClasses])  # the expected outputs, labels

# Take the input to Fourier domain
CurrentInput = tf.cast(InputData, tf.complex64)
CurrentInput = tf.transpose(CurrentInput, [3, 0, 1, 2])
fourierInput = tf.fft2d(CurrentInput, name=None)
fourierInput = tf.transpose(fourierInput, [1, 2, 3, 0])
fourierInput = tf.roll(fourierInput, shift=[int(Size[0] // 2), int(Size[1] // 2)], axis=[1, 2])
CurrentFilters = Size[-1]

# a loop which creates all layers
for N in range(len(NumKernels)):
    with tf.variable_scope('conv' + str(N)):
        fourierInput = convolution_in_freq_domain_without_ifft(fourierInput, NumKernels[N])

with tf.variable_scope('FC'):
    fourierInput = tf.square(tf.real(fourierInput)) + tf.square(tf.imag(fourierInput))
    CurrentShape = fourierInput.get_shape()
    FeatureLength = int(CurrentShape[1] * CurrentShape[2] * CurrentShape[3])
    FC = tf.reshape(fourierInput, [-1, FeatureLength])
    W = tf.get_variable('W', [FeatureLength, NumClasses])
    FC = tf.matmul(FC, W)
    Bias = tf.get_variable('Bias', [NumClasses])
    FC = tf.add(FC, Bias)

with tf.name_scope('loss'):
    Loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=OneHotLabels, logits=FC))

with tf.name_scope('optimizer'):
    Optimizer = tf.train.AdamOptimizer(LearningRate).minimize(Loss)

with tf.name_scope('accuracy'):
    CorrectPredictions = tf.equal(tf.argmax(FC, 1), tf.argmax(OneHotLabels, 1))
    Accuracy = tf.reduce_mean(tf.cast(CorrectPredictions, tf.float32))

Init = tf.global_variables_initializer()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.compat.v1.Session(config=config) as Sess:
    Sess.run(Init)
    Step = 1
    while Step < NumIteration:
        UsedInBatch = random.sample(range(features_train.shape[0]), BatchLength)
        batch_xs = features_train[UsedInBatch, :]
        batch_ys = labels_train_onehot[UsedInBatch, :]
        batch_xs = np.reshape(batch_xs, [BatchLength] + Size)
        _, Acc, L = Sess.run([Optimizer, Accuracy, Loss], feed_dict={InputData: batch_xs, OneHotLabels: batch_ys})
        if (Step % 100) == 0:
            print("Iteration: " + str(Step))
            print("Accuracy:" + str(Acc))
            print("Loss:" + str(L))
        if (Step % EvalFreq) == 0:
            SumAcc = 0.0
            for i in range(0, features_test.shape[0]):
                batch_xs = features_test[i, :]
                batch_ys = labels_test_onehot[i, :]
                batch_xs = np.reshape(batch_xs, [1] + Size)
                batch_ys = np.reshape(batch_ys, [1, NumClasses])
                a = Sess.run(Accuracy, feed_dict={InputData: batch_xs, OneHotLabels: batch_ys})
                SumAcc += a
            print("Independent Test set: " + str(float(SumAcc) / features_test.shape[0]))
        Step += 1

    # Después de imprimir la forma de la salida
    print("Forma de la salida:", FC.shape)

    # Visualizar algunas imágenes y sus predicciones
    num_visualize = 5  # Puedes ajustar el número de imágenes a visualizar

    # Seleccionar aleatoriamente algunas muestras del conjunto de prueba
    visualize_indices = np.random.choice(features_test.shape[0], num_visualize, replace=False)

    # Crear una sesión sin inicializar explícitamente las variables
    for i in visualize_indices:
        # Obtener la imagen y la etiqueta verdadera
        input_image = features_test[i, :]
        true_label = labels_test_onehot[i, :]

        # Preprocesar la entrada para que sea compatible con la red neuronal
        input_image = np.reshape(input_image, [1] + Size)

        # Obtener la predicción del modelo
        prediction = np.argmax(Sess.run(FC, feed_dict={InputData: input_image}), axis=1)

        # Visualizar la imagen, la etiqueta verdadera y la predicción
        plt.imshow(np.squeeze(input_image), cmap='gray')
        plt.title(f'Etiqueta real: {np.argmax(true_label)}, Etiqueta predicha: {prediction[0]}')
        plt.show()
        plt.pause(2)  # Pausa de 2 segundos entre imágenes
